# 4. Using XGBoost in Pipelines

These are my personal notes of the Datacamp course [Extreme Gradient Boosting with XGBoost](https://app.datacamp.com/learn/courses/extreme-gradient-boosting-with-xgboost).

The course has 4 main sections:

1. Classification
2. Regression
3. Fine-tuning XGBoost
4. **Using XGBoost in Pipelines**: the current notebook.

XGBoost is an implementation of the [Gradient Boosting](https://en.wikipedia.org/wiki/Gradient_boosting) algorithm in C++ which has bindings to other languages, such as Python. It has the following properties:

- Fast.
- Best performance.
- Parallelizable, on a computer and across the network. So it can work with huge datasets distributed on several nodes/GPUs.
- We can use it for classification and regression.
- The [Python API](https://xgboost.readthedocs.io/en/stable/python/python_api.html) is easy to use and has two major flavors or sub-APIs:
  - The **Scikit-Learn API**: We instantiate `XGBRegressor()` or `XGBClassifier` and then we can `fit()` and `predict()`, using the typical Scikit-Learn parameters; we can even use those objects with other Scikit-Learn modules, such as `GridSearchCV`.
  - The **Learning API**: The native XGBoost Python API requires to convert the dataframes into `DMatrix` objects first; then, we have powerful methods which allow for tuning many parameters: `xgb.cv()`, `xgb.train()`. The native/learning API is very easy to use. **Note: the parameter names are different compared to the Scikit-Learn API!**

Classification is the original supervised learning problem addressed by XGBoost, although it can also handle regression problems.

### Installation

```python
# PIP
pip install xgboost

# Conda: General
conda install -c conda-forge py-xgboost

# Conda: CPU only
conda install -c conda-forge py-xgboost-cpu

# Conda: Use NVIDIA GPU: Linux x86_64
conda install -c conda-forge py-xgboost-gpu

# For tree visualization
pip install graphviz
```

### Table of Contents

- [4.1 Example of Pipelines](#4.1-Example-of-Pipelines)
- [4.2 Data Processing in Pipelines](#4.2-Data-Processing-in-Pipelines)
    - Categoricals: LabelEncoder
    - Categoricals: OneHotEncoder
    - Pipeline
    - Cross-Validation
- [4.3 Full Pipeline Example: Kidney Disease Dataset](#4.3-Full-Pipeline-Example:-Kidney-Disease-Dataset)
- [4.4 Housing Pipeline with Random Search Cross-Validation](#4.4-Housing-Pipeline-with-Random-Search-Cross-Validation)

## 4.1 Example of Pipelines

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("../data/ames_housing_trimmed_processed.csv")
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [3]:
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,Remodeled,GrLivArea,BsmtFullBath,BsmtHalfBath,...,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,PavedDrive_P,PavedDrive_Y,SalePrice
0,60,65.0,8450,7,5,2003,0,1710,1,0,...,0,0,0,0,1,0,0,0,1,208500
1,20,80.0,9600,6,8,1976,0,1262,0,1,...,0,1,0,0,0,0,0,0,1,181500
2,60,68.0,11250,7,5,2001,1,1786,1,0,...,0,0,0,0,1,0,0,0,1,223500
3,70,60.0,9550,7,5,1915,1,1717,1,0,...,0,0,0,0,1,0,0,0,1,140000
4,60,84.0,14260,8,5,2000,0,2198,1,0,...,0,0,0,0,1,0,0,0,1,250000


In [4]:
rf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("rf_model", RandomForestRegressor())])

scores = cross_val_score(rf_pipeline,
                         X,
                         y,
                         scoring="neg_mean_squared_error",
                         cv=10)

## 4.2 Data Processing in Pipelines

In [7]:
df = pd.read_csv("../data/ames_unprocessed_data.csv")

In [8]:
df.shape

(1460, 21)

In [9]:
df.isna().sum()

MSSubClass        0
MSZoning          0
LotFrontage     259
LotArea           0
Neighborhood      0
BldgType          0
HouseStyle        0
OverallQual       0
OverallCond       0
YearBuilt         0
Remodeled         0
GrLivArea         0
BsmtFullBath      0
BsmtHalfBath      0
FullBath          0
HalfBath          0
BedroomAbvGr      0
Fireplaces        0
GarageArea        0
PavedDrive        0
SalePrice         0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSSubClass    1460 non-null   int64  
 1   MSZoning      1460 non-null   object 
 2   LotFrontage   1201 non-null   float64
 3   LotArea       1460 non-null   int64  
 4   Neighborhood  1460 non-null   object 
 5   BldgType      1460 non-null   object 
 6   HouseStyle    1460 non-null   object 
 7   OverallQual   1460 non-null   int64  
 8   OverallCond   1460 non-null   int64  
 9   YearBuilt     1460 non-null   int64  
 10  Remodeled     1460 non-null   int64  
 11  GrLivArea     1460 non-null   int64  
 12  BsmtFullBath  1460 non-null   int64  
 13  BsmtHalfBath  1460 non-null   int64  
 14  FullBath      1460 non-null   int64  
 15  HalfBath      1460 non-null   int64  
 16  BedroomAbvGr  1460 non-null   int64  
 17  Fireplaces    1460 non-null   int64  
 18  GarageArea    1460 non-null 

### Categoricals: LabelEncoder

In [11]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Fill missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == "object")

# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()

# Print the head of the categorical columns
print(df[categorical_columns].head())

# Create LabelEncoder object: le
le = LabelEncoder()

# Apply LabelEncoder to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))

# Print the head of the LabelEncoded categorical columns
print(df[categorical_columns].head())

  MSZoning Neighborhood BldgType HouseStyle PavedDrive
0       RL      CollgCr     1Fam     2Story          Y
1       RL      Veenker     1Fam     1Story          Y
2       RL      CollgCr     1Fam     2Story          Y
3       RL      Crawfor     1Fam     2Story          Y
4       RL      NoRidge     1Fam     2Story          Y
   MSZoning  Neighborhood  BldgType  HouseStyle  PavedDrive
0         3             5         0           5           2
1         3            24         0           2           2
2         3             5         0           5           2
3         3             6         0           5           2
4         3            15         0           5           2


### Categoricals: OneHotEncoder

In [12]:
# Import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Create OneHotEncoder: ohe
ohe = OneHotEncoder(sparse=False)

# Apply OneHotEncoder to categorical columns - output is no longer a dataframe: df_encoded
df_encoded = ohe.fit_transform(df)

# Print first 5 rows of the resulting dataset - again, this will no longer be a pandas dataframe
print(df_encoded[:5, :])

# Print the shape of the original DataFrame
print(df.shape)

# Print the shape of the transformed array
print(df_encoded.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1460, 21)
(1460, 3369)


### Categoricals: DictVectorizer

In [14]:
# Import DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# Convert df into a dictionary: df_dict
# Each row becomes a dictionary with key = col name and value = cell value
# All dictionaries/rows are inside a list
df_dict = df.to_dict(orient="records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False)

# Apply dv on df: df_encoded
df_encoded = dv.fit_transform(df_dict)

# Print the resulting first five rows
print(df_encoded[:5,:])

# Print the vocabulary
# NOTE: the order of the columns is not preserved!
# The dv.vocabulary_ maps the column index of df_encoded to a column name
# but the original order is not preserved.
# Thus, the df_encoded matrix is compliant with dv.vocabulary_ but the column
# order is new.
print(dv.vocabulary_)

[[3.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 2.000e+00 5.480e+02
  1.710e+03 1.000e+00 5.000e+00 8.450e+03 6.500e+01 6.000e+01 3.000e+00
  5.000e+00 5.000e+00 7.000e+00 2.000e+00 0.000e+00 2.085e+05 2.003e+03]
 [3.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 2.000e+00 4.600e+02
  1.262e+03 0.000e+00 2.000e+00 9.600e+03 8.000e+01 2.000e+01 3.000e+00
  2.400e+01 8.000e+00 6.000e+00 2.000e+00 0.000e+00 1.815e+05 1.976e+03]
 [3.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 2.000e+00 6.080e+02
  1.786e+03 1.000e+00 5.000e+00 1.125e+04 6.800e+01 6.000e+01 3.000e+00
  5.000e+00 5.000e+00 7.000e+00 2.000e+00 1.000e+00 2.235e+05 2.001e+03]
 [3.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 1.000e+00 6.420e+02
  1.717e+03 0.000e+00 5.000e+00 9.550e+03 6.000e+01 7.000e+01 3.000e+00
  6.000e+00 5.000e+00 7.000e+00 2.000e+00 1.000e+00 1.400e+05 1.915e+03]
 [4.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 2.000e+00 8.360e+02
  2.198e+03 1.000e+00 5.000e+00 1.426e+04 8.400e+01 6.000e+0

In [18]:
df_encoded.shape

(1460, 21)

In [16]:
len(df_dict)

1460

In [17]:
df_dict[:2]

[{'MSSubClass': 60,
  'MSZoning': 3,
  'LotFrontage': 65.0,
  'LotArea': 8450,
  'Neighborhood': 5,
  'BldgType': 0,
  'HouseStyle': 5,
  'OverallQual': 7,
  'OverallCond': 5,
  'YearBuilt': 2003,
  'Remodeled': 0,
  'GrLivArea': 1710,
  'BsmtFullBath': 1,
  'BsmtHalfBath': 0,
  'FullBath': 2,
  'HalfBath': 1,
  'BedroomAbvGr': 3,
  'Fireplaces': 0,
  'GarageArea': 548,
  'PavedDrive': 2,
  'SalePrice': 208500},
 {'MSSubClass': 20,
  'MSZoning': 3,
  'LotFrontage': 80.0,
  'LotArea': 9600,
  'Neighborhood': 24,
  'BldgType': 0,
  'HouseStyle': 2,
  'OverallQual': 6,
  'OverallCond': 8,
  'YearBuilt': 1976,
  'Remodeled': 0,
  'GrLivArea': 1262,
  'BsmtFullBath': 0,
  'BsmtHalfBath': 1,
  'FullBath': 2,
  'HalfBath': 0,
  'BedroomAbvGr': 3,
  'Fireplaces': 1,
  'GarageArea': 460,
  'PavedDrive': 2,
  'SalePrice': 181500}]

### Pipeline

Here, everything is done in a pipeline. Note that we need to use the Scikit-Learn API in order to pack the XGBoost models intoa pipeline.

In [29]:
import xgboost as xgb

In [28]:
df = pd.read_csv("../data/ames_housing_trimmed_processed.csv")
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [30]:
# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor())]

# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

# Fit the pipeline
xgb_pipeline.fit(X.to_dict("records"), y)

Pipeline(steps=[('ohe_onestep', DictVectorizer(sparse=False)),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

### Cross-Validation

In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [34]:
df = pd.read_csv("../data/ames_housing_trimmed_processed.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [36]:
# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor(max_depth=2, objective="reg:squarederror"))]

# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

# Cross-validate the model
cross_val_scores = cross_val_score(xgb_pipeline,
                         X.to_dict("records"),
                         y,
                         cv=10,
                         scoring="neg_mean_squared_error")

# Print the 10-fold RMSE
print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

10-fold RMSE:  27683.04157118635


## 4.3 Full Pipeline Example: Kidney Disease Dataset

We need to use the Scikit-Learn API in order to pack the XGBoost models intoa pipeline.

In this section, the [chronic kidney disease dataset](https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease) is used, which requirems more data processing.

From the dataset web:

```
1.Age(numerical) 
age in years 
2.Blood Pressure(numerical) 
bp in mm/Hg 
3.Specific Gravity(nominal) 
sg - (1.005,1.010,1.015,1.020,1.025) 
4.Albumin(nominal) 
al - (0,1,2,3,4,5) import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import xgboost as xgb
5.Sugar(nominal) 
su - (0,1,2,3,4,5) 
6.Red Blood Cells(nominal) 
rbc - (normal,abnormal) 
7.Pus Cell (nominal) 
pc - (normal,abnormal) 
8.Pus Cell clumps(nominal) 
pcc - (present,notpresent) 
9.Bacteria(nominal) 
ba - (present,notpresent) 
10.Blood Glucose Random(numerical) 
bgr in mgs/dl 
11.Blood Urea(numerical) 
bu in mgs/dl 
12.Serum Creatinine(numerical) 
sc in mgs/dl 
13.Sodium(numerical) 
sod in mEq/L 
14.Potassium(numerical) 
pot in mEq/L 
15.Hemoglobin(numerical) 
hemo in gms 
16.Packed Cell Volume(numerical) 
17.White Blood Cell Count(numerical) 
wc in cells/cumm 
18.Red Blood Cell Count(numerical) 
rc in millions/cmm 
19.Hypertension(nominal) 
htn - (yes,no) 
20.Diabetes Mellitus(nominal) 
dm - (yes,no) 
21.Coronary Artery Disease(nominal) 
cad - (yes,no) 
22.Appetite(nominal) 
appet - (good,poor) 
23.Pedal Edema(nominal) 
pe - (yes,no) 
24.Anemia(nominal) 
ane - (yes,no) 
25.Class (nominal) 
class - (ckd,notckd)
```

In [198]:
# Import necessary modules
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn_pandas import DataFrameMapper
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
#from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [159]:
column_names = ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'class']

In [160]:
#!pip install sklearn-pandas

In [161]:
# The CSV has no column names, but I got them from the web/Datacamp
# Also, note that NaN values are marked with a ?
df = pd.read_csv("../data/chronic_kidney_disease.csv", names=column_names, na_values='?')

In [162]:
df.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [163]:
df.shape

(400, 25)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   bgr     248 non-null    object 
 6   bu      335 non-null    object 
 7   sc      396 non-null    object 
 8   sod     396 non-null    object 
 9   pot     356 non-null    float64
 10  hemo    381 non-null    float64
 11  pcv     383 non-null    float64
 12  wc      313 non-null    float64
 13  rc      312 non-null    float64
 14  rbc     348 non-null    float64
 15  pc      329 non-null    float64
 16  pcc     294 non-null    float64
 17  ba      269 non-null    float64
 18  htn     398 non-null    object 
 19  dm      398 non-null    object 
 20  cad     398 non-null    object 
 21  appet   399 non-null    object 
 22  pe

In [192]:
# Extract features and target
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [193]:
y.unique()

array(['ckd', 'notckd'], dtype=object)

In [166]:
# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
                                            [([numeric_feature], SimpleImputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True,
                                            df_out=True
                                           )

# Apply categorical imputer
categorical_imputation_mapper = DataFrameMapper(
                                                [([category_feature], SimpleImputer(strategy="most_frequent")) for category_feature in categorical_columns],
                                                input_df=True,
                                                df_out=True
                                               )

age        9
bp        12
sg        47
al        46
su        49
bgr      152
bu        65
sc         4
sod        4
pot       44
hemo      19
pcv       17
wc        87
rc        88
rbc       52
pc        71
pcc      106
ba       131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64


In [182]:
# Custom transformer which converts a dataframe into a dictionary
class Dictifier(BaseEstimator, TransformerMixin):
    """Convert dataset to a dictionary."""

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = pd.DataFrame(X)
        return df.to_dict("records")

In [183]:
# Combine the numeric and categorical transformations
numeric_categorical_union = FeatureUnion([
                                          ("num_mapper", numeric_imputation_mapper),
                                          ("cat_mapper", categorical_imputation_mapper)
                                         ])

In [216]:
# Create full pipeline
pipeline = Pipeline([
                     ("featureunion", numeric_categorical_union),
                     ("dictifier", Dictifier()),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3, objective="binary:logistic", use_label_encoder=False))
                    ])

In [217]:
# Transform labels
le = preprocessing.LabelEncoder()
y = le.fit_transform(y).astype("int")

In [218]:
cross_val_scores = cross_val_score(pipeline,
                         X,
                         y,
                         scoring="roc_auc",
                         cv=10)

[14:22:58] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:22:58] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:22:58] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:22:59] WARNING: /Users/runner/miniforge3/cond

In [219]:
cross_val_scores

array([1.   , 1.   , 1.   , 1.   , 0.992, 1.   , 1.   , 1.   , 1.   ,
       1.   ])

## 4.4 Housing Pipeline with Random Search Cross-Validation

In [239]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

data = pd.read_csv("../data/ames_housing_trimmed_processed.csv")
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [240]:
data.shape

(1460, 57)

In [242]:
X.shape

(1460, 56)

In [243]:
y.shape

(1460,)

In [244]:
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,Remodeled,GrLivArea,BsmtFullBath,BsmtHalfBath,...,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,PavedDrive_P,PavedDrive_Y,SalePrice
0,60,65.0,8450,7,5,2003,0,1710,1,0,...,0,0,0,0,1,0,0,0,1,208500
1,20,80.0,9600,6,8,1976,0,1262,0,1,...,0,1,0,0,0,0,0,0,1,181500
2,60,68.0,11250,7,5,2001,1,1786,1,0,...,0,0,0,0,1,0,0,0,1,223500
3,70,60.0,9550,7,5,1915,1,1717,1,0,...,0,0,0,0,1,0,0,0,1,140000
4,60,84.0,14260,8,5,2000,0,2198,1,0,...,0,0,0,0,1,0,0,0,1,250000


In [245]:
xgb_pipeline = Pipeline([("st_scaler", StandardScaler()),
                         ("xgb_model", xgb.XGBRegressor())])
gbm_param_grid = {
    'xgb_model__subsample': np.arange(.05, 1, .05),
    'xgb_model__max_depth': np.arange(3,20,1),
    'xgb_model__colsample_bytree': np.arange(.1,1.05,.05) }

randomized_neg_mse = RandomizedSearchCV(estimator=xgb_pipeline,
                                        param_distributions=gbm_param_grid, 
                                        n_iter=10,
                                        scoring='neg_mean_squared_error', cv=4)

randomized_neg_mse.fit(X, y)

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('st_scaler', StandardScaler()),
                                             ('xgb_model',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           enable_categorical=False,
                                                           gamma=None,
                                                           gpu_id=None,
                                                           importance_type=None,
                                                           interaction_constraints=None,
                                            

In [248]:
print("Best rmse: ", np.sqrt(np.abs(randomized_neg_mse.best_score_)))

Best rmse:  29969.312787260624


In [249]:
print("Best model: ", randomized_neg_mse.best_estimator_)

Best model:  Pipeline(steps=[('st_scaler', StandardScaler()),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.25000000000000006,
                              enable_categorical=False, gamma=0, gpu_id=-1,
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=3, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.8500000000000001,
                              tree_method='exact', validate_parameters=1,
                 